In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Concatenate,Dense,Input,Embedding,Bidirectional,LSTM,GRU
import keras.backend as K
from tensorflow.keras.utils import to_categorical

In [2]:
imdb_df = pd.read_csv("/home/rohit/datasets/IMDB_Dataset.csv")

In [3]:
d = {'negative':0,'positive':1}

In [4]:
imdb_df['sentiment'] = imdb_df['sentiment'].apply(lambda x : d.get(x))

In [5]:
X_train,X_test,y_train,y_test = train_test_split(imdb_df['review'],imdb_df['sentiment'],test_size=0.2)

In [6]:
X_train = X_train.to_numpy()

In [7]:
X_test = X_test.to_numpy()

In [8]:
y_train = y_train.to_numpy()

In [9]:
y_test = y_test.to_numpy()

In [10]:
#clean_text
import re
def clean_text(text,punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~''',
    stop_words=['the', 'a', 'and', 'is', 'be', 'will']):
    
    
    text = re.sub(r'https?://\S+|www\.\S+', '',text)
    text = re.sub(r'<.*?>', '',text)
    # Removing the punctuations
    for x in text.lower(): 
        if x in punctuations: 
            text = text.replace(x, "")
            
    text = text.lower()
    # Removing stop words
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    
    
    text = re.sub(r'\s+', ' ',text).strip()
    return text

In [11]:
X_train = [clean_text(x) for x in X_train]

In [12]:
X_test = [clean_text(x) for x in X_test]

In [13]:
t=Tokenizer()
t.fit_on_texts(X_train)
text_matrix=t.texts_to_sequences(X_train)

In [14]:
test_matrix = t.texts_to_sequences(X_test)

In [15]:
vocab_length = len(t.word_index) + 1

In [17]:
from keras.preprocessing.sequence import pad_sequences
text_pad = pad_sequences(text_matrix, maxlen=128, padding='post')

In [25]:
test_pad = pad_sequences(test_matrix, maxlen=128, padding='post')

In [19]:
from keras.layers import Layer
import keras.backend as K

In [20]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [21]:
import keras

In [22]:
inputs=Input((128,))
x=Embedding(input_dim=vocab_length,output_dim=100,input_length=128,\
            embeddings_regularizer=keras.regularizers.l2(.001))(inputs)
att_in=LSTM(100,return_sequences=True,dropout=0.3,recurrent_dropout=0.2)(x)
att_out=attention()(att_in)
outputs=Dense(1,activation='sigmoid',trainable=True)(att_out)
model=keras.Model(inputs,outputs)
model.summary()

2022-08-24 04:08:32.188252: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 128, 100)          19241100  
_________________________________________________________________
lstm (LSTM)                  (None, 128, 100)          80400     
_________________________________________________________________
attention (attention)        (None, 100)               228       
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 19,321,829
Trainable params: 19,321,829
Non-trainable params: 0
_________________________________________________________________


2022-08-24 04:08:32.195995: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-24 04:08:32.196464: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-24 04:08:32.197522: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-24 04:08:32.197953: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [23]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(text_pad,y_train,batch_size=256,epochs=8,verbose=1,shuffle=True,validation_split=0.2)

2022-08-24 04:08:38.779283: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/8
125/125 [==============================] - 15s 107ms/step - loss: 2.3952 - acc: 0.7266 - val_loss: 0.4823 - val_acc: 0.8420
Epoch 2/8
125/125 [==============================] - 13s 106ms/step - loss: 0.4647 - acc: 0.8624 - val_loss: 0.4777 - val_acc: 0.8500
Epoch 3/8
125/125 [==============================] - 13s 105ms/step - loss: 0.4391 - acc: 0.8795 - val_loss: 0.4865 - val_acc: 0.8544
Epoch 4/8
125/125 [==============================] - 13s 106ms/step - loss: 0.4311 - acc: 0.8867 - val_loss: 0.4692 - val_acc: 0.8634
Epoch 5/8
125/125 [==============================] - 13s 105ms/step - loss: 0.4135 - acc: 0.8947 - val_loss: 0.4664 - val_acc: 0.8660
Epoch 6/8
125/125 [==============================] - 13s 106ms/step - loss: 0.3947 - acc: 0.9050 - val_loss: 0.4758 - val_acc: 0.8645
Epoch 7/8
125/125 [==============================] - 13s 106ms/step - loss: 0.3781 - acc: 0.9134 - val_loss: 0.4785 - val_acc: 0.8694
Epoch 8/8
125/125 [==============================] - 13s 107ms

In [26]:
model.evaluate(test_pad,y_test)

313/313 [==============================] - 6s 18ms/step - loss: 0.4824 - acc: 0.8705


[0.482359915971756, 0.8705000281333923]